In [1]:
# Import Libraries
from PIL import Image
import numpy as np
import random
import hashlib

In [2]:
s_box = [
    0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76,
    0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0,
    0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15,
    0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75,
    0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84,
    0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF,
    0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8,
    0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2,
    0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73,
    0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB,
    0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79,
    0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08,
    0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A,
    0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E,
    0xE1, 0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF,
    0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16
]


In [3]:
rcon = [0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40, 0x80, 0x1B, 0x36]

Steps to do this Project
1. ECC for Key Management
2. AES for Image Encryption

Steps
Generate and Exchange the AES key with ECC

1. Generate a random AES Key and an ECC Key pair for the sender and the recipient
2. Use an ECC-based protocol to securely share the AES encryption key with the recipient. The sender encrypts the AES key using ECC and sends it to the recipient.
3. The recipient decrypts the received ECC-encrypted AES key to obtain the original AES key, which is used for image encryption and decryption

Encrypt the Medical Image with Modified AES
1. Load the image, convert it to grayscale or RGB, and flatten it into a 1D byte array.
2. Do the AES encryption on the image
3. Store or transmit the encrypted image as a new file.

Decrypting the medical image.
1. The recipient uses the ECC to decrypt the AES key if it was encrypted during transmission.
2. Use AES key to decrypt the image data.
3. Remove padding from the data.
4. Convert the decrypted byte array back into its original shape and save or display it.

In [4]:
# Curve25519
a = 0; b = 7

G = (55066263022277343669578718895168534326250603453777594175500187360389116729240, 
     32670510020758816978083085130507043184471273380659243275938904335757337482424) 

p = pow(2, 256) - pow(2, 32) - pow(2, 9) - pow(2, 8) - pow(2, 7) - pow(2, 6) - pow(2, 4) - pow(2, 0)
n = 115792089237316195423570985008687907852837564279074904382605163141518161494337

In [5]:
def add_points(P, Q, p):
    x1, y1 = P
    x2, y2 = Q
     
    if x1 == x2 and y1 == y2:
        beta = (3*x1*x2 + a) * pow(2*y1, -1, p)
    else:
        beta = (y2 - y1) * pow(x2 - x1, -1, p)
     
    x3 = (beta*beta - x1 - x2) % p
    y3 = (beta * (x1 - x3) - y1) % p
     
    is_on_curve((x3, y3), p)
         
    return x3, y3
 
def is_on_curve(P, p):
    x, y = P
    assert (y*y) % p == ( pow(x, 3, p) + a*x + b ) % p
     
def apply_double_and_add_method(G, k, p):
    target_point = G
     
    k_binary = bin(k)[2:] #0b1111111001
     
    for i in range(1, len(k_binary)):
        current_bit = k_binary[i: i+1]
         
        # doubling - always
        target_point = add_points(target_point, target_point, p)
         
        if current_bit == "1":
            target_point = add_points(target_point, G, p)
     
    is_on_curve(target_point, p)
     
    return target_point

In [6]:
#  alice private key
ka = random.getrandbits(256)
 
# bob private key
kb = random.getrandbits(256)
 
# alice public key
Qa = apply_double_and_add_method(G = G, k = ka, p = p)
 
# bob public key
Qb = apply_double_and_add_method(G = G, k = kb, p = p)

In [7]:
# alice performs this calculation
Sa = apply_double_and_add_method(G = Qb, k = ka, p = p)
 
# bob performs this calculation
Sb = apply_double_and_add_method(G = Qa, k = kb, p = p)
 
if (Sa == Sb):
    print("Shared secrect key is same")
    print(Sa)

Shared secrect key is same
(27146355196110330085266105395923211870048733723066016641104769530481715676081, 60605776833992123869333049748344853680797285096628428869773436337030354295452)


In [8]:
x, y = Sa
x_bytes = x.to_bytes((x.bit_length()+7)//8, byteorder='big')
y_bytes = y.to_bytes((y.bit_length()+7)//8, byteorder='big')

combined_bytes = x_bytes + y_bytes

full_hash = hashlib.sha256(combined_bytes).digest()
aes_encrypted_key = full_hash[:16]
print(aes_encrypted_key.hex())

f6d1cf06c98a74bcc9c59b8ffbc0b44e


In [9]:
def get_image_size(image_path):
    with Image.open(image_path) as img:
        width, height = img.size
    return width, height

In [10]:
def image_to_128bit_blocks(image_path):
    with Image.open(image_path) as img:
        img = img.convert("L")
        img_data = np.array(img)
        byte_data = img_data.tobytes()

    block_size = 16
    blocks = [byte_data[i:i+block_size] for i in range(0, len(byte_data), block_size)]

    if len(blocks[-1]) < block_size:
        length_of_padding = b'\x00' * (block_size - len(blocks[-1]))
        blocks[-1] = blocks[-1] + length_of_padding
    return blocks, length_of_padding

In [11]:
def reconstruct_image_from_blocks(blocks, original_size, block_size=16):
    byte_data = b''.join(blocks)
    required_bytes = original_size[0] * original_size[1]
    byte_data = byte_data[:required_bytes]  # Trim any excess padding bytes
    img_array = np.frombuffer(byte_data, dtype=np.uint8)
    img_array = img_array.reshape(original_size[1], original_size[0])
    img = Image.fromarray(img_array, 'L')  # 'L' for grayscale
    return img

In [12]:
image_path = "C:\\MyFolder\\Projects\\cns\\casestudy\\test.png"
width, height = get_image_size(image_path)
print(f"Image size: {width} X {height} pixels")
blocks, length_of_padding = image_to_128bit_blocks(image_path)
for i, block in enumerate(blocks):
    print(f"Block{i+1}: {block.hex()}")

# print("Length of Padding: ", len(length_of_padding))

Image size: 600 X 599 pixels
Block1: ffffffffffffffffffffffffffffffff
Block2: ffffffffffffffffffffffffffffffff
Block3: ffffffffffffffffffffffffffffffff
Block4: ffffffffffffffffffffffffffffffff
Block5: ffffffffffffffffffffffffffffffff
Block6: ffffffffffffffffffffffffffffffff
Block7: ffffffffffffffffffffffffffffffff
Block8: ffffffffffffffffffffffffffffffff
Block9: ffffffffffffffffffffffffffffffff
Block10: ffffffffffffffffffffffffffffffff
Block11: ffffffffffffffffffffffffffffffff
Block12: ffffffffffffffffffffffffffffffff
Block13: ffffffffffffffffffffffffffffffff
Block14: ffffffffffffffffffffffffffffffff
Block15: ffffffffffffffffffffffffffffffff
Block16: ffffffffffffffffffffffffffffffff
Block17: ffffffffffffffffffffffffffffffff
Block18: ffffffffffffffffffffffffffffffff
Block19: ffffffffffffffffffffffffffffffff
Block20: ffffffffffffffffffffffffffffffff
Block21: ffffffffffffffffffffffffffffffff
Block22: ffffffffffffffffffffffffffffffff
Block23: ffffffffffffffffffffffffffffffff
Block24: fffff

In [13]:
def rot_word(word):
    return word[1:]+word[:1]

In [14]:
def sub_word(word):
    return bytes([s_box[b] for b in word])

In [15]:
def key_expansion(key):
    key_columns = 4
    total_columns = 4*(10+1)
    round_keys = [key[i:i+4] for i in range(0, len(key), 4)]
    for i in range(key_columns, total_columns):
        temp = round_keys[i-1]
        if i % key_columns == 0:
            temp = sub_word(rot_word(temp))
            temp = bytes([temp[0]^rcon[(i//key_columns)-1]]+list(temp[1:]))
            
        round_keys.append(bytes([round_keys[i-key_columns][j]^temp[j] for j in range(4)]))

    expanded_key = [b''.join(round_keys[i:i + 4]) for i in range(0, len(round_keys), 4)]
    return expanded_key

In [16]:
expanded_key = key_expansion(aes_encrypted_key)
print("Original Key:", aes_encrypted_key.hex())
for i, round_key in enumerate(expanded_key):
    print(f"Round Key {i + 1}: {round_key.hex()}")

Original Key: f6d1cf06c98a74bcc9c59b8ffbc0b44e
Round Key 1: f6d1cf06c98a74bcc9c59b8ffbc0b44e
Round Key 2: 4d5ce00984d694b54d130f3ab6d3bb74
Round Key 3: 29b67247ad60e6f2e073e9c856a052bc
Round Key 4: cdb617f660d6f10480a518ccd6054a70
Round Key 5: ae604600ceb6b7044e13afc89816e5b8
Round Key 6: f9b92a46370f9d42791c328ae10ad732
Round Key 7: beb709be89b894fcf0a4a67611ae7144
Round Key 8: 1a14123c93ac86c0630820b672a651f2
Round Key 9: bec59b7c2d691dbc4e613d0a3cc76cf8
Round Key 10: 6395da974efcc72b009dfa213c5a96d9
Round Key 11: eb05ef7ca5f92857a564d276993e44af


In [17]:
print("Round1Key: ", expanded_key[0].hex())

Round1Key:  f6d1cf06c98a74bcc9c59b8ffbc0b44e


In [18]:
def block_to_state(block):
    state = [list(block[i:i+4]) for i in range(0,16,4)]
    return state

In [19]:
def state_to_bytes(state):
    return bytes([state[i][j] for i in range(4) for j in range(4)])

In [20]:
# state = block_to_state(block0)
# for row in state:
#     print(row)

In [21]:
# round1key = expanded_key[0]
# print(round1key)

In [22]:
# round1key = block_to_state(round1key)
# for row in round1key:
#     print(row)

In [23]:
# round1key = state_to_bytes(round1key)
# print(round1key)

# AES Transformation Function

In [24]:
def sub_bytes(state):
    return [[s_box[state[i][j]] for j in range(4)] for i in range(4)]

In [25]:
def shift_rows(state):
    for i in range(1, 4):
        state[i] = state[i][i:] + state[i][:i]
    return state

In [26]:
def mix_columns(state):
    def mix_single_column(column):
        r = [
            (column[0] << 1) ^ ((column[0] & 0x80) >> 7) * 0x1B ^ column[1] ^ column[2] ^ column[3],
            column[0] ^ (column[1] << 1) ^ ((column[1] & 0x80) >> 7) * 0x1B ^ column[2] ^ column[3],
            column[0] ^ column[1] ^ (column[2] << 1) ^ ((column[2] & 0x80) >> 7) * 0x1B ^ column[3],
            column[0] ^ column[1] ^ column[2] ^ (column[3] << 1) ^ ((column[3] & 0x80) >> 7) * 0x1B
        ]
        return [v & 0xFF for v in r]
    
    # Apply the MixColumns transformation column by column
    for i in range(4):
        state[i] = mix_single_column(state[i])
    return state

In [27]:
def add_round_key(state, round_key):
    return[[state[i][j] ^ round_key[i][j] for j in range(4)] for i in range(4)]

In [28]:
# Aes Encryption
def aes_encrypt(block, expanded_key):
    state = block_to_state(block)
    state = add_round_key(state, block_to_state(expanded_key[0]))

    for i in range(1, 10):
        state = sub_bytes(state)
        state = shift_rows(state)
        state = mix_columns(state)
        state = add_round_key(state, block_to_state(expanded_key[i]))

    state = sub_bytes(state)
    state = shift_rows(state)
    state = add_round_key(state, block_to_state(expanded_key[10]))
    return state_to_bytes(state)


In [29]:
test_block = blocks[0]

In [30]:
aes_encrypted_key

b'\xf6\xd1\xcf\x06\xc9\x8at\xbc\xc9\xc5\x9b\x8f\xfb\xc0\xb4N'

In [31]:
expanded_key = key_expansion(aes_encrypted_key)
expanded_key[0]

b'\xf6\xd1\xcf\x06\xc9\x8at\xbc\xc9\xc5\x9b\x8f\xfb\xc0\xb4N'

In [32]:
print(test_block.hex())

ffffffffffffffffffffffffffffffff


In [33]:
test_encrypted_block = aes_encrypt(test_block, expanded_key)

In [34]:
print(test_encrypted_block)

b"ll\x95\xa1\xc9k,x\x80\xd8\xc4\xf8'\xe0\xee\xb8"


In [35]:
print(test_encrypted_block.hex())

6c6c95a1c96b2c7880d8c4f827e0eeb8


In [36]:
encrypted_blocks = []

for i in range(len(blocks)):
    encrypted_blocks.append(aes_encrypt(blocks[i], expanded_key))

In [37]:
for i, block in enumerate(encrypted_blocks):
    print(f"Block{i+1}: {block.hex()}")

Block1: 6c6c95a1c96b2c7880d8c4f827e0eeb8
Block2: 6c6c95a1c96b2c7880d8c4f827e0eeb8
Block3: 6c6c95a1c96b2c7880d8c4f827e0eeb8
Block4: 6c6c95a1c96b2c7880d8c4f827e0eeb8
Block5: 6c6c95a1c96b2c7880d8c4f827e0eeb8
Block6: 6c6c95a1c96b2c7880d8c4f827e0eeb8
Block7: 6c6c95a1c96b2c7880d8c4f827e0eeb8
Block8: 6c6c95a1c96b2c7880d8c4f827e0eeb8
Block9: 6c6c95a1c96b2c7880d8c4f827e0eeb8
Block10: 6c6c95a1c96b2c7880d8c4f827e0eeb8
Block11: 6c6c95a1c96b2c7880d8c4f827e0eeb8
Block12: 6c6c95a1c96b2c7880d8c4f827e0eeb8
Block13: 6c6c95a1c96b2c7880d8c4f827e0eeb8
Block14: 6c6c95a1c96b2c7880d8c4f827e0eeb8
Block15: 6c6c95a1c96b2c7880d8c4f827e0eeb8
Block16: 6c6c95a1c96b2c7880d8c4f827e0eeb8
Block17: 6c6c95a1c96b2c7880d8c4f827e0eeb8
Block18: 6c6c95a1c96b2c7880d8c4f827e0eeb8
Block19: 6c6c95a1c96b2c7880d8c4f827e0eeb8
Block20: 6c6c95a1c96b2c7880d8c4f827e0eeb8
Block21: 6c6c95a1c96b2c7880d8c4f827e0eeb8
Block22: 6c6c95a1c96b2c7880d8c4f827e0eeb8
Block23: 6c6c95a1c96b2c7880d8c4f827e0eeb8
Block24: 6c6c95a1c96b2c7880d8c4f827e0eeb8
B

In [38]:
width, height = get_image_size("C:\\MyFolder\\Projects\\cns\\casestudy\\test.png")
original_size = (width, height)

In [39]:
encrypted_image = reconstruct_image_from_blocks(encrypted_blocks, original_size)

In [40]:
encrypted_image.save("Encrypted_image.png")